# 第2回　ロジスティック回帰
3月17日　石村

## データセット
乳がんデータ（二値分類）

In [48]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
cancer = load_breast_cancer()

X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = cancer.target

In [6]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [10]:
y.sum() / len(y)

0.6274165202108963

## sklearnのロジスティック回帰

In [70]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='none')
lr.fit(X[X.columns[:3]], y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='none',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [73]:
lr.coef_

array([[ 6.02624306, -0.24276883, -1.06751971]])

In [74]:
lr.intercept_

array([18.32558117])

In [17]:
y_pred = lr.predict_proba(X)

In [22]:
y_pred[3]

array([0.1530617, 0.8469383])

In [32]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

sigmoid(X.values @ lr.coef_.T + lr.intercept_)

array([[4.15024168e-15],
       [2.88387744e-08],
       [2.34333842e-07],
       [8.46938304e-01],
       [4.46939838e-06],
       [2.72151146e-01],
       [1.17038573e-06],
       [3.27258393e-02],
       [3.10805674e-01],
       [1.96548876e-02],
       [1.39947086e-03],
       [1.36354084e-05],
       [6.13900711e-06],
       [8.75601807e-01],
       [5.47278099e-01],
       [2.79335096e-03],
       [1.94369851e-04],
       [2.92977573e-05],
       [1.22817899e-15],
       [9.67038529e-01],
       [9.95582872e-01],
       [9.99615696e-01],
       [6.28245922e-02],
       [8.13991080e-16],
       [5.62759991e-15],
       [8.05258596e-08],
       [2.28025077e-02],
       [5.35028897e-06],
       [4.67099976e-07],
       [2.66137543e-03],
       [6.41552137e-10],
       [1.03714861e-04],
       [3.10953724e-05],
       [6.24456174e-08],
       [6.03476983e-04],
       [9.63021130e-04],
       [8.31136924e-02],
       [9.99740648e-01],
       [6.37990046e-01],
       [8.07108911e-01],


## 自分で実装ver

In [37]:
import numpy as np
class MyLogisticRegression():
    def __init__(self, fit_intercept=True):
        self.w = None
        self.fit_intercept = None
        self.fit_intercept=fit_intercept
        return

In [34]:
X_add = X.copy()
X_add['const'] = 1

In [75]:
import statsmodels.api as sm

slr = sm.Logit(y, sm.add_constant(X[X.columns[:3]]))
lr_result = slr.fit(method='lbfgs', max_itr=100, penalty='none')
print(lr_result.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  569
Model:                          Logit   Df Residuals:                      565
Method:                           MLE   Df Model:                            3
Date:                Tue, 17 Mar 2020   Pseudo R-squ.:                  0.6581
Time:                        16:45:19   Log-Likelihood:                -128.46
converged:                      False   LL-Null:                       -375.72
                                        LLR p-value:                7.344e-107
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             10.9665      1.252      8.762      0.000       8.513      13.420
mean radius        8.1029      1.009      8.031      0.000       6.125      10.080
mean texture      -0.1760      0.038